In [220]:
import numpy as np
from difflib import *
import os
import pandas as pd

In [221]:

import warnings
warnings.simplefilter('ignore')
# We will display plots right inside Jupyter Notebook
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime

import re
import jieba
import mysql.connector
import matplotlib.font_manager as mfm

In [222]:
def get_connect():
    conn_temp = mysql.connector.connect(
            host = '139.162.122.115',#本地地址
            user = 'travel',#一般默认用户名
            passwd = 'ksd53890045',#本地数据库登录密码（这里用你自己的密码！！！）
            db = 'travelbigdata',#数据库名称（这里用你自己的数据库名称！！！）
            port = 3306,#安装mysql默认的端口号
            charset = 'utf8'#设置数据库统一编码  
    )
    return conn_temp

In [223]:
tablename='google'
table={
    'id':'int(11) not null auto_increment PRIMARY KEY',
    'icon':'varchar(200)',
    'lon':'DOUBLE not null',
    'lat':'DOUBLE not null',
    'name':'varchar(100)',
    'placeid':  'varchar(100) not null  COMMENT "goole查詢用"' ,
    'rating':'float',
    'review':'float',
    'price':'varchar(100)',
    'url':'varchar(1024)',
    'types':'varchar(1024)',
    'phone':'varchar(100)',
    'vicinity':'varchar(1024)',
    'googletype':'varchar(100)',
    'imageUrl': 'varchar(1024)',
    'webUrl':'varchar(1024)',
    'area':'varchar(100) ' ,
    'city':'varchar(100) ' ,
    'fbcount': 'int(11) COMMENT "FB統計數量"',
    'tripcount': 'int(11) COMMENT "trip統計數量"',
    'create_at':'datetime   NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT "創建時間"',
    'modify_at':' datetime  NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT "修改時間"'
    }
othersql='UNIQUE (placeid)'

In [224]:
def insetDB(tablename,table,data,conn_temp):
    cursor = conn_temp.cursor()
    sqltmpname=''
    sqltmpvalue=''
    for i in data:
        sqltmpname+=str(i)+' '+','
        if('char' in table[i]):
            sqltmpvalue+='"%s",' %data[i]
        else:
            sqltmpvalue+='%s,' %data[i]
    sqltmpname=sqltmpname[:-1]
    sqltmpvalue=sqltmpvalue[:-1]
    sqldata=[i for i in data]
    sqlInsert='insert into  '+tablename+'('+sqltmpname+') VALUES ('+sqltmpvalue+')'        
        
    
    sqlUpdate='UPDATE '+tablename+'  set rating=%s,review=%s,area="%s",city="%s" where placeid= "%s" ' %\
        (data['rating'],data['review'],data['area'],data['city'],data['placeid'])
    
    
    
    try:
        cursor.execute(sqlInsert)
        conn_temp.commit()
    except:
        print('exsit   '+str(data['placeid']))
        #print(sqlUpdate)
        cursor.execute(sqlUpdate)
        conn_temp.commit()
    return 

In [225]:
#### define all types in google data ,remove thoat not in this types
foodtypes=['餐廳', '餐館', '咖啡', '熟食', '冰品', '飲料', '餃子', '牛扒', '果汁', 
      '早餐', '酒吧',  '拉麵', '居酒屋', '酒店', '雪糕', '甜品', '麵店', '麵包', 
      '冷麵', '麵食店', '美式扒房','熟食店', '珍珠奶茶', '包點', '壽司', 
      '烏冬專門店', '菜館', '三文治店', '薄餅', '曲奇餅店',  '酒樓', '海味店', 
      '餡餅店', '快餐', '酒家', '飲料店', '茶館', '餅店', '健康食品專賣店','燒烤', 
      '小食店','餐車店','餐廳','茶藝']
nottypes=['用品店','批發商']

In [226]:
###   檢查是否存在上述的foodtypes， 如果存在於nottypes 將會去除  避免ㄧ些怪怪的
def checkFood(onetype,types):
    boFood=False
    for t in types:
        if(t in onetype):
            boFood=True
            break
    return boFood
def checkNotFood(onetype,types):
    boFood=True
    for t in types:
        if(t in onetype):
            boFood=False
            break
    return boFood    

In [227]:
############################
#
#
#
#資料讀取  字的要改名阿!!
#
#
#
############################

In [228]:
### 讀取資料!!!
namearea='Xinxing'

In [229]:
dataGoogleApi=pd.read_csv('data/googleApi/'+namearea+'.csv')
dataGoogleInfo=pd.read_csv('data/googleInfo/'+namearea+'.csv')

In [230]:
dataGoogleApi.head(2)

,from,icon,id,lat,lon,name,placeid,price,rating,type,user_ratings_total,vicinity
0,google,https://maps.gstatic.com/mapfiles/place_api/ic...,0648bea0a2602ed8324f6208ca8b63cb4d86c6f5,22.627278,120.301435,高雄市,ChIJG3R6elFDbjQRNypzVEqiJkg,NaN,NaN,"['locality', 'administrative_area_level_1', 'p...",NaN,None
1,google,https://maps.gstatic.com/mapfiles/place_api/ic...,94cda2ead1679304161a638ab16366a5b77a0a4a,22.627360,120.297403,高雄市立大同醫院圖書室,ChIJI_0QUX0EbjQRDFz16qw7img,NaN,3.6,"['library', 'point_of_interest', 'establishment']",54.0,前金區中華三路68號4F


In [231]:
dataGoogleInfo.head(2)

,cidUrl,googletype,imageUrl,opentime,phone,placeid,rating,review,webUrl
0,NaN,NaN,https://lh5.googleusercontent.com/p/AF1QipNS3A...,NaN,NaN,ChIJG3R6elFDbjQRNypzVEqiJkg,NaN,NaN,NaN
1,NaN,超級市場,//geo0.ggpht.com/cbk?cb_client=maps_sv.tactile...,NaN,NaN,ChIJoRjQeooEbjQRqXA2zk_ybeQ,NaN,NaN,NaN


In [232]:
dataGoogle=pd.merge(dataGoogleApi,dataGoogleInfo,on='placeid')

In [233]:
dataGoogle.to_csv('data/googleMerage/'+namearea+'.csv')

In [234]:
dataGoogle.columns

Index(['from', 'icon', 'id', 'lat', 'lon', 'name', 'placeid', 'price',
       'rating_x', 'type', 'user_ratings_total', 'vicinity', 'cidUrl',
       'googletype', 'imageUrl', 'opentime', 'phone', 'rating_y', 'review',
       'webUrl'],
      dtype='object')

In [235]:
dataGoogle.columns=['from', 'icon', 'id', 'lat', 'lon', 'name', 'placeid', 'price',
       'rating', 'type', 'user_ratings_total', 'vicinity', 'url',
       'googletype', 'imageUrl', 'opentime', 'phone', 'ray', 'review',
       'webUrl']

In [236]:
##### 滿足 googletype  為上述的成分
#### 滿足 types 裡面有 food 的因子

boCheck=[]
for i,j in zip(dataGoogle['googletype'],dataGoogle['type']):
    boTmp=False
    if( checkFood(str(i),foodtypes) and  checkNotFood(str(i),nottypes) ):
        boTmp=True
    if('food' in str(j)):
        boTmp=True
    boCheck.append(boTmp)

In [237]:
#### 取出  food 的內容
food=dataGoogle[boCheck].reset_index(drop=True)

In [238]:
### 定義一下高雄區域，利用地址分類資料  ######
kao=['鼓山區','三民區','新興區','前金區','苓雅區','鹽埕區','前鎮區','旗津區','左營區','鳳山區','大寮區','大樹區','鳥松區','小港區','楠梓區','林園區','岡山區','仁武區','大社區','旗山區']
place='高雄'

In [239]:
area=[]
city=[]
for st,i in enumerate(food['vicinity']):
    areatmp=None
    placetmp=None
    for j in kao:
        if('鳳山市' in i):
            areatmp='鳳山區'
            placetmp=place            
        if(j in i):
            areatmp=j
            placetmp=place
    if(pd.isnull(areatmp)):print(i)
    area.append(areatmp)
    city.append(placetmp)
food['area']=area
food['city']=city

中正四路71號
高雄市
南台路128號
No. 163-1, Linsen 1st Road, Xinxing District
3F, No, No. 267, 林森一路


In [240]:
tablename='google'
table={
    'id':'int(11) not null auto_increment PRIMARY KEY',
    'icon':'varchar(200)',
    'lon':'DOUBLE not null',
    'lat':'DOUBLE not null',
    'name':'varchar(100)',
    'placeid':  'varchar(100) not null  COMMENT "goole查詢用"' ,
    'rating':'float',
    'review':'float',
    'price':'varchar(100)',
    'url':'varchar(1024)',
    'types':'varchar(1024)',
    'phone':'varchar(100)',
    'vicinity':'varchar(1024)',
    'googletype':'varchar(100)',
    'imageUrl': 'varchar(1024)',
    'webUrl':'varchar(1024)',
    'area':'varchar(100) ' ,
    'city':'varchar(100) ' ,
    'fbcount': 'int(11) COMMENT "FB統計數量"',
    'tripcount': 'int(11) COMMENT "trip統計數量"',
    'create_at':'datetime   NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT "創建時間"',
    'modify_at':' datetime  NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT "修改時間"'
    }
othersql='UNIQUE (placeid)'

In [241]:
food.columns

Index(['from', 'icon', 'id', 'lat', 'lon', 'name', 'placeid', 'price',
       'rating', 'type', 'user_ratings_total', 'vicinity', 'url', 'googletype',
       'imageUrl', 'opentime', 'phone', 'ray', 'review', 'webUrl', 'area',
       'city'],
      dtype='object')

In [242]:
conn_temp=get_connect()
for st in range(len(food)):
    if(st<=-1):
        pass
    else:
        rating='NULL' if pd.isna(food['rating'][st])else food['rating'][st]
        review='NULL' if pd.isna(food['user_ratings_total'][st])else food['user_ratings_total'][st]
        name=str(food['name'][st]).replace('|','').replace('、','').replace(',','').replace(':','')
        types=str(food['type'][st]).replace(',','_')
        vicinity=str(food['vicinity'][st]).replace(',','')
        data={
            'icon':food['icon'][st],
            'lon':food['lon'][st],
            'lat':food['lat'][st],
            'name':name,
            'rating':rating,
            'review':review,
            'price':food['price'][st],
            'url':food['url'][st],
            'types':types,
            'phone':food['phone'][st],
            'vicinity':food['vicinity'][st],
            'googletype':food['googletype'][st],
            'imageUrl':food['imageUrl'][st],
            'webUrl':food['webUrl'][st],
            'placeid':food['placeid'][st],
            'area':food['area'][st],
            'city':food['city'][st]
        }
        insetDB('google',table,data,conn_temp)
conn_temp.close()

exsit   ChIJbb0s2GEEbjQRnOpa-q2Jtuk
exsit   ChIJq68CgYoEbjQRL6rJnIKN4C0
exsit   ChIJJ6gtdooEbjQRXfxOWT4w9do
exsit   ChIJaeFid4oEbjQRo8j_4V-1Sfg
exsit   ChIJB3zGeIoEbjQRZIuYvS4LaSk
exsit   ChIJC2hUeIoEbjQRX0RGmTeOfmA
exsit   ChIJ7QTIgIgEbjQR85KM8Pw-HWs
exsit   ChIJ-WCmgIgEbjQR2K0M___RX4s
exsit   ChIJPSQUUIgEbjQRxSHGqiQFhzE
exsit   ChIJixmUXIgEbjQRgqltxmKPn7o
exsit   ChIJmQAPQogEbjQRdVRkMWiCWBE
exsit   ChIJE-JtUIgEbjQRyF-G6A6z2po
exsit   ChIJ-0n3r4kEbjQRqpZlRR6Bip4
exsit   ChIJJ1nDu4kEbjQRdiJd5II6wvo
exsit   ChIJ0eqhvIkEbjQRYEk7xE_Dg9A
exsit   ChIJ0eqhvIkEbjQRc2xOXJdi0tA
exsit   ChIJdYqxvIkEbjQRjenEPKqzssg
exsit   ChIJSzFnu4kEbjQRpx0ecoLQvak
exsit   ChIJkYQBsIkEbjQRR-1r73H3h3c
exsit   ChIJTU1fToEEbjQRYzT082Tv5xk
exsit   ChIJJYgVR4EEbjQR69m5wZzsRXE
exsit   ChIJwY-SVIEEbjQRsPa8U-mdHiY
exsit   ChIJ-4J0T4EEbjQRtJZn7R2YBKs
exsit   ChIJE5OQroYEbjQRyvTry4gD7CY
exsit   ChIJN8Y3NoEEbjQRLNxiIR7a-tM
exsit   ChIJdw9_T4EEbjQRHa43Z2O1vjA
exsit   ChIJI6BX84YEbjQRVcf_D-3UbvQ
exsit   ChIJidrewYYEbjQRKI62

In [65]:
bo=[ True if 'facebook' in  str(i) else False for i in food['webUrl']]

In [68]:
food[bo].reset_index(drop=True).to_csv(namearea+'_FB.csv')